In [1]:
import os
import sys
import tarfile
import gif
import numpy as np
import igraph as ig
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_settings_v3 import *
from dismantling import get_index_list
from planar import create_proximity_graph, distance
from auxiliary import get_base_network_name

fig_dir = '../draft/figs'
param = 'param'

attack_dict = {
    'Ran': r'$\mathrm{Rnd}$', 'Deg': r'$\mathrm{ID}$', 'DegU': r'$\mathrm{RD}$',
    'Btw': r'$\mathrm{IB}$', 'BtwU': r'$\mathrm{RB}$', 
    'Eigenvector': r'$\mathrm{IE}$', 'EigenvectorU': r'$\mathrm{RE}$',
    'CI': r'$\mathrm{ICI}$', 'CIU': r'$\mathrm{RCI}$', 'CIU2': r'$\mathrm{RCI2}$',
}

In [32]:
from random import randint

@gif.frame
def plot(x, y):
    plt.figure(figsize=(5, 3), dpi=100)
    plt.scatter(x, y)
    plt.xlim((0, 100))
    plt.ylim((0, 100))

frames = []
for _ in range(50):
    x = [randint(0, 100) for _ in range(10)]
    y = [randint(0, 100) for _ in range(10)]
    frame = plot(x, y)
    frames.append(frame)
    
gif.save(frames, "./chaos.gif", duration=100)

In [110]:
def get_position(net_type, size, net_dir=None):

    if net_type == 'Lattice':  
        L = size
        position = np.array([[i//L, i%L] for i in range(L*L)])
    else:
        N = size
        L = np.sqrt(N)
        ## Extract network file
        pos_tar_file_name = os.path.join(net_dir, 'position.tar.gz')
        tar = tarfile.open(pos_tar_file_name, 'r:gz')
        tar.extractall(net_dir)
        tar.close()

        pos_file_name = os.path.join(net_dir, 'position.txt')
        position = np.loadtxt(pos_file_name) * L
        os.remove(pos_file_name)
    
    return position

def get_max_pos(dir_name):
    
    tar_input_name = 'comp_data.tar.gz'
    full_tar_input_name = os.path.join(dir_name, tar_input_name)
    tar = tarfile.open(full_tar_input_name, 'r:gz')
    tar.extractall(dir_name)
    tar.close()

    full_file_name  = os.path.join(dir_name, 'comp_data.txt')
    aux = np.loadtxt(full_file_name)
    os.remove(full_file_name)

    Ngcc_values = aux[:,0][::-1] 
    delta_values = np.abs(np.diff(Ngcc_values))
    max_pos = np.argmax(delta_values)
    delta_max = delta_values[max_pos]
    
    return max_pos, delta_max

def load_data(net_type, size, param, attack, seed):
    
    dir_name = os.path.join('../networks', net_type)
    base_net_name, base_net_name_size = get_base_network_name(net_type, size, param)
    net_name = base_net_name_size + '_{:05d}'.format(seed)
    base_net_dir = os.path.join(dir_name, base_net_name, base_net_name_size)
    net_dir = os.path.join(base_net_dir, net_name)
    
    attack_dir_name = os.path.join(base_net_dir, net_name, attack)
    oi_file_name = os.path.join(attack_dir_name, 'oi_list.txt')
    index_list = np.loadtxt(oi_file_name, dtype='int')

    ## Extract network file
    tar_input_name = net_name + '.tar.gz'
    full_tar_input_name = os.path.join(net_dir, tar_input_name)
    tar = tarfile.open(full_tar_input_name, 'r:gz')
    tar.extractall(net_dir)
    tar.close()

    input_name = net_name + '.txt'
    full_input_name = os.path.join(net_dir, input_name)

    ## Read network file
    g = ig.Graph().Read_Edgelist(full_input_name, directed=False)

    ## Remove network file
    os.remove(full_input_name)
    
    g.vs['oi'] = range(g.vcount())
    g.vs['position'] = get_position(net_type, size, net_dir)
    g.vs['attack_index'] = index_list
    
    max_pos, delta_max = get_max_pos(attack_dir_name)
    
    return g, max_pos, delta_max


L = 64
N = L*L
attack = 'BtwU'

size = {
    'Lattice': L,
    'DT': N
}
network = 'Lattice'
#network = 'DT'
seed = 0

#print(delta_max, delta_max*L, delta_max*L*L)

In [ ]:
@gif.frame
def plot(g, th, index_list, position, label_dict):
    
    marker = 'o'
    
    plt.figure(figsize=(10,10))

    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.xticks = range(L)
    plt.yticks = range(L)   
    plt.xlim(-1, L+1)
    plt.ylim(-1, L+1)
    #plt.margins(0.5)
    
    for idx in index_list[:th]:
        plt.scatter(*position[idx], s=50, color='k', marker=marker)
    for idx in index_list[th:]:
        comm_label = g.vs[g.vs['oi'].index(idx)]['membership']
        plt.scatter(*position[idx], s=50, color=colors[label_dict[comm_label]], marker=marker) 
        
frames = []
thresholds = np.arange(max_pos+2)
for th in thresholds:
#for th in range(max_pos, max_pos+2):

    g, max_pos, delta_max = load_data(network, size[network], param, attack, seed)
    
    index_list = g.vs['attack_index']
    position = g.vs['position']
    g.delete_vertices(index_list[:th])

    gcc = g.components(mode='WEAK').giant()
    membership = np.array(g.components(mode='WEAK').membership)
    g.vs['membership'] = membership
    
    number, count = np.unique(membership, return_counts=True)
    comm_labels = number[np.argsort(count)[::-1]] 
    label_dict = dict([(label, i) for i, label in enumerate(comm_labels)])
    
    number_max =  number[0]
    gcc_members = np.where(membership==number_max)[0]
    gcc_members_values = [g.vs[index]['oi'] for index in gcc_members]

    frame = plot(g, th, index_list, position, label_dict)
    frames.append(frame)
    
    if th == thresholds[-1]:
        frames += [frame] * 10
        
gif.save(frames, '{}_L{}_seed{:05}.gif'.format(network, L, seed), duration=500)
print('Finish')